We can either get a omega matrix by:
- a covariance matrix of our alphas, or
- a diagonal matrix like brandon said

we will calculate a diagonal matrix like brandon said which will be calculated as:
the entries are [(1/SR_s) * q_s]^2

where q_s is the signal expected return matrix (a 3x1 matrix for a day equal to w_s * alpha_s)

In [25]:
import sf_quant.data as sfd
import polars as pl
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt

In [26]:
# Signal Returns:

bab_return = pl.read_parquet("/home/jaic007/silverFund/combineSignal/signalCalc/signalReturns/signal_bab.parquet")
mom_return = pl.read_parquet("/home/jaic007/silverFund/combineSignal/signalCalc/signalReturns/signal_momentum.parquet")
meanrev_return = pl.read_parquet("/home/jaic007/silverFund/combineSignal/signalCalc/signalReturns/signal_meanrev.parquet")

In [27]:
# Q Matrix

qMatrix = pl.read_parquet("/home/jaic007/silverFund/combineSignal/BL/data/qMatrix.parquet")

In [28]:
def compute_rolling_sharpe(df: pl.DataFrame, ret_col: str, window: int = (22*12)) -> pl.DataFrame:
    """
    Compute rolling Sharpe ratio for one signal.
    """
    return (
        df.sort("date")
          .with_columns([
              pl.col(ret_col).rolling_mean(window).alias("mean"),
              pl.col(ret_col).rolling_std(window).alias("std")
          ])
          .with_columns([
              (pl.col("mean") / pl.col("std")).alias(f"{ret_col}_sharpe")
          ])
          .select(["date", f"{ret_col}_sharpe"])
    )

In [29]:
bab_sr = compute_rolling_sharpe(bab_return, "bab_return")
mom_sr = compute_rolling_sharpe(mom_return, "momentum_return")
meanrev_sr = compute_rolling_sharpe(meanrev_return, "meanrev_return")

In [ ]:
# Join all of our signals calculated rolling sharpes
rolling_sharpes = (
    bab_sr.join(mom_sr, on="date", how="full", suffix="_mom")
          .join(meanrev_sr, on="date", how="full", suffix="_meanrev")
          .sort("date")
          .drop(['date_mom', 'date_meanrev'])
)

In [ ]:
# Join our rolling sharpes with qMatrix 
q_with_sr = qMatrix.join(rolling_sharpes, on="date", how="inner")

In [32]:
q_with_sr

date,bab_er,momentum_er,meanrev_er,bab_return_sharpe,momentum_return_sharpe,meanrev_return_sharpe
date,f64,f64,f64,f64,f64,f64
1995-07-03,-0.002372,0.0,0.0,null,null,null
1995-07-05,-0.002456,0.0,0.0,null,null,null
1995-07-06,-0.002549,0.0,0.0,null,null,null
1995-07-07,-0.002443,0.0,0.0,null,null,null
1995-07-10,-0.002684,0.0,0.0,null,null,null
…,…,…,…,…,…,…
2024-12-24,0.003805,0.030739,0.045066,0.102995,0.083353,0.012108
2024-12-26,0.003904,0.03071,0.042249,0.097064,0.073532,0.008225
2024-12-27,0.003647,0.02877,0.043792,0.092093,0.068387,0.006793


In [ ]:
# Compute Omega diagonals as said by brandon 
omega_diag = (
    q_with_sr
    .with_columns([
        ((pl.col("bab_er") / pl.col("bab_return_sharpe"))**2).alias("omega_bab"),
        ((pl.col("momentum_er") / pl.col("momentum_return_sharpe"))**2).alias("omega_momentum"),
        ((pl.col("meanrev_er") / pl.col("meanrev_return_sharpe"))**2).alias("omega_meanrev"),
    ])
    .select(["date", "omega_bab", "omega_momentum", "omega_meanrev"])
)

In [34]:
omega_diag

date,omega_bab,omega_momentum,omega_meanrev
date,f64,f64,f64
1995-07-03,null,null,null
1995-07-05,null,null,null
1995-07-06,null,null,null
1995-07-07,null,null,null
1995-07-10,null,null,null
…,…,…,…
2024-12-24,0.001365,0.135997,13.852293
2024-12-26,0.001618,0.174418,26.382974
2024-12-27,0.001569,0.176987,41.554983
